### Thai - Registry : Scraping translated data

### https://porta.fda.moph.go.th/FDA_SEARCH_ALL/MAIN/SEARCH_CENTER_MAIN.aspx

In [1]:
#!pip install plyer
#from plyer import notification
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import geckodriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from datetime import datetime as date
import pandas as pd, numpy as np
import warnings as ws
ws.filterwarnings("ignore")
import re, os, time, glob
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pickle
from typing import List
# Create directory if it doesn't exist
directory = "Fetched Data"
if not os.path.exists(directory):
    os.makedirs(directory)
# Initialize Firefox with options
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
geckodriver_autoinstaller.install()


def clean_str(string):
    string = string.replace('\xa0', " ").strip()
    string = string.replace('\n', "")
    string = string.replace('\t', "")
    return string.strip()

Firefox version:  122.0
Available geckodriver version:  v0.34.0


In [40]:
## Use IVD Gateway Major player to search MDA using company names
major_players = pd.read_csv("IVD_gateway_major_players_actual.csv")
major_players 
players = major_players.player_name.values.tolist()
## Split out company names delimited by '-'
keywords = []
for name in players:
    if '-' in name:
        spread_names = name.split('-')
        for i in spread_names:
            keywords.append(i.strip())
    else:
        keywords.append(name)

In [41]:
len(set(keywords))
keywords = list(set(keywords))

In [ ]:
print("Searching FDA TH using keywords...")
catch = dict()
for keyword in keywords[:5]:
    collector = []
    print(keyword, end=" ")
    try:
        driver, total_results = load_search(keyword, driver)
        if not total_results = 'No records'
            flag, table = extract_page_data(driver)
            ## find next button until
            total_res = total_results.text.strip().split(" ")[0]
            page_count = total_res / 50
            for page in range(page_count+1):
                wait = WebDriverWait(driver, 10)
                next_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'rgPageNext')))
                next_btn.click()
                table = extract_page_data(driver)
                time.sleep(1)
    except:
        print("Error")
        
    if len(collector):
        catch[keyword] = collector
    
    

In [69]:
def load_search(keyword, driver):
    driver = webdriver.Firefox(options=options)
    driver.get('https://porta.fda.moph.go.th/FDA_SEARCH_ALL/MAIN/SEARCH_CENTER_MAIN.aspx')
    driver.maximize_window()
    time.sleep(2)
    try:
        pop_close = driver.find_element(By.XPATH,'//*[@id="myModal2"]/div/div/div[3]/button')
        pop_close.click()
    except Exception as e:
        print(e, keyword)
    #driver.find_element(By.XPATH , "//div[@class = 'modal-content']/div[@class='modal-footer']/button[@class = 'btn btn-default']").click()
    time.sleep(4)

    driver.find_element(By.XPATH , "//div[@class = 'col-lg-2 col-md-2']/table/tbody/tr/td/input[@id = 'ContentPlaceHolder1_R_LIST']").click()
    time.sleep(4)

    driver.find_element(By.ID , 'ContentPlaceHolder1_CheckBoxList1').find_element(By.XPATH , "//tbody/tr[7]").click()
    time.sleep(4)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    wait = WebDriverWait(driver, 10)
    search_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_btn_search"]')))
    input_box = driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_txt_search"]')
    input_box.send_keys(keyword)
    driver.execute_script("arguments[0].click();", search_btn)
    time.sleep(2)
    wait = WebDriverWait(driver, 10)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    flag, table = extract_page_data(driver)
    if flag == 'No results':
        return driver, 'No records'
    else:
        totals = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_RadGrid1_ctl00_ctl03_ctl01_PageSizeComboBox_Input"]')))
        totals.click()
        fifty = driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_RadGrid1_ctl00_ctl03_ctl01_PageSizeComboBox_DropDown"]/div/ul/li[3]')
        fifty.click()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        total_res_count = driver.find_element(By.CLASS_NAME,'rgInfoPart')
        total_res_count = total_res_count.text

        return driver, total_res_count
    
def extract_page_data(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source)
    data = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'rgMasterTable')))
    table = soup.find('table',{'class':'rgMasterTable'})
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    if 'No records to display.' in table.text:
        return 'No results', table
    else:
        table = pd.read_html(str(table))[0]
        return 'Found', table   
        

In [65]:
driver, count = load_search('roche',driver)

In [87]:
driver.quit()

In [2]:
## Consolidaton
chunks = glob.glob(os.path.join(os.getcwd(), "*FDA*.xlsx"))
chunks

['C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\FDA TH\\FDA TH.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\FDA TH\\TH FDA 200.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\FDA TH\\TH FDA 300.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\FDA TH\\TH FDA 50.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\FDA TH\\TH FDA 50200.xlsx']

In [3]:
chunks = [pd.read_excel(chunk) for chunk in chunks]

In [4]:
grill = pd.concat(chunks[1:])
del grill['Unnamed: 0']
del grill['Unnamed: 9']


In [5]:
grill

,Product type,CAT_NO,Certificate/License,Product name Thai - English,Trade name,Licensee name,Product details,Newcode,status
0,medical equipment,20300900,65-2-2-1-0012106,Werfens HemosIL VWF:RCo HemosIL VWF:RCo,HemosIL VWF:RCo,P.C.N. Healthcare Company Limited,-,U1MN000102656323570000011C,remain
1,เครื่องมือแพทย์,06-6361-20,66-1-3-2-0000433,คีมสำหรับผ่าตัดเนื้อเยื่ออ่อน คล้ายกรรไกร ใช้ซ...,"Surgical soft-tissue manipulation forceps, sci...",บริษัท เซอร์จีกซ์ เซอร์จิเคิล จำกัด,Wiener Modell 20 cm (1PC/1BAG),U1MC000102661280276000484484C,คงอยู่
2,เครื่องมือแพทย์,06-6361-22,66-1-3-2-0000433,คีมสำหรับผ่าตัดเนื้อเยื่ออ่อน คล้ายกรรไกร ใช้ซ...,"Surgical soft-tissue manipulation forceps, sci...",บริษัท เซอร์จีกซ์ เซอร์จิเคิล จำกัด,Wiener Modell 22 cm (1PC/1BAG),U1MC000102661280276000485485C,คงอยู่
3,เครื่องมือแพทย์,06-6361-25,66-1-3-2-0000433,คีมสำหรับผ่าตัดเนื้อเยื่ออ่อน คล้ายกรรไกร ใช้ซ...,"Surgical soft-tissue manipulation forceps, sci...",บริษัท เซอร์จีกซ์ เซอร์จิเคิล จำกัด,Wiener Modell 25 cm (1PC/1BAG),U1MC000102661280276000486486C,คงอยู่
4,เครื่องมือแพทย์,06-6363-22,66-1-3-2-0000433,คีมสำหรับผ่าตัดเนื้อเยื่ออ่อน คล้ายกรรไกร ใช้ซ...,"Surgical soft-tissue manipulation forceps, sci...",บริษัท เซอร์จีกซ์ เซอร์จิเคิล จำกัด,Wiener Modell(Walzl) 22 cm (1PC/1BAG),U1MC000102661280276000487487C,คงอยู่
...,...,...,...,...,...,...,...,...,...
2867,medical equipment,VL2610DB,USA6308966,Reliant Pencil Holster 15' (4.6 m) Reliant Pen...,Reliant Pencil Holster 15' (4.6 m),Medtronic (Thailand) Company Limited,Reliant Pencil Holster 15' (4.6 m),U1MC0001026350004000249196C,expire
2868,medical equipment,VL2610E,USA6308966,"Reliant Pencil Erbe, Holster 15' (4.6 m) Relia...","Reliant Pencil Erbe, Holster 15' (4.6 m)",Medtronic (Thailand) Company Limited,"Reliant Pencil Erbe, Holster 15' (4.6 m)",U1MC0001026350004000250197C,expire
2869,medical equipment,VL2615,USA6308966,"Reliant Pencil Coated Blade, Holster 10' (3 m)...","Reliant Pencil Coated Blade, Holster 10' (3 m)",Medtronic (Thailand) Company Limited,"Reliant Pencil Coated Blade, Holster 10' (3 m)",U1MC0001026350004000251198C,expire
2870,medical equipment,VL2615DB,USA6308966,"Reliant Pencil Coated Blade, Holster 15' (4.6 ...","Reliant Pencil Coated Blade, Holster 15' (4.6 m)",Medtronic (Thailand) Company Limited,"Reliant Pencil Coated Blade, Holster 15' (4.6 m)",U1MC0001026350004000252199C,expire


In [6]:
grill.columns = ['Product_type', 'CAT_NO', 'Cert_License', 'Device Description',
       'Device Name', 'Company', 'Product details', 'New_Codes',
       'Status']

In [7]:
chunks[0].columns

Index(['Product_type', 'CAT_NO', 'Cert_License', 'Device Name',
       'Device Description', 'Company', 'Product details', 'New_Codes',
       'Status'],
      dtype='object')

In [8]:
chunks[0].columns

Index(['Product_type', 'CAT_NO', 'Cert_License', 'Device Name',
       'Device Description', 'Company', 'Product details', 'New_Codes',
       'Status'],
      dtype='object')

In [9]:
th_fda = pd.concat([chunks[0], grill])
th_fda.shape

(72020, 9)

In [10]:
th_fda = th_fda.drop_duplicates()

In [15]:
th_fda.to_excel('TH MDR 29jan24.xlsx')

In [11]:
th_fda

,Product_type,CAT_NO,Cert_License,Device Name,Device Description,Company,Product details,New_Codes,Status
0,medical equipment,None50,USA5805703,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC00010200727000004040C,expire
1,medical equipment,None182,USA5904658,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,DRX-ASCEND SYSTEM Accessories,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC000102007320000259259C,expire
2,medical equipment,13-1641VASL,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000133133C,expire
3,medical equipment,13-1641VASL-3,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000134134C,expire
4,medical equipment,13-1641,DEU5805277,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC0001020103400017001700C,expire
...,...,...,...,...,...,...,...,...,...
2817,medical equipment,VL2610DB,USA6308966,Reliant Pencil Holster 15' (4.6 m),Reliant Pencil Holster 15' (4.6 m) Reliant Pen...,Medtronic (Thailand) Company Limited,Reliant Pencil Holster 15' (4.6 m),U1MC0001026350004000249196C,expire
2818,medical equipment,VL2610E,USA6308966,"Reliant Pencil Erbe, Holster 15' (4.6 m)","Reliant Pencil Erbe, Holster 15' (4.6 m) Relia...",Medtronic (Thailand) Company Limited,"Reliant Pencil Erbe, Holster 15' (4.6 m)",U1MC0001026350004000250197C,expire
2819,medical equipment,VL2615,USA6308966,"Reliant Pencil Coated Blade, Holster 10' (3 m)","Reliant Pencil Coated Blade, Holster 10' (3 m)...",Medtronic (Thailand) Company Limited,"Reliant Pencil Coated Blade, Holster 10' (3 m)",U1MC0001026350004000251198C,expire
2820,medical equipment,VL2615DB,USA6308966,"Reliant Pencil Coated Blade, Holster 15' (4.6 m)","Reliant Pencil Coated Blade, Holster 15' (4.6 ...",Medtronic (Thailand) Company Limited,"Reliant Pencil Coated Blade, Holster 15' (4.6 m)",U1MC0001026350004000252199C,expire


In [44]:
import re
df = th_fda.copy()
# Regular expression for matching thai characters
thai_pattern = r'[\u0E00-\u0E7F]+'

# Function to check for the presence of Korean characters
def contains_thai(text):
    if pd.notna(text):
        if isinstance(text, str):
            if re.search(thai_pattern, text):
                return True
            return False

# Apply the function to the DataFrame column and get the rows with Korean characters
df['to_translate'] = df['Product details'].map(lambda x: contains_korean(x))

df

,Product_type,CAT_NO,Cert_License,Device Name,Device Description,Company,Product details,New_Codes,Status,to_translate
0,medical equipment,None50,USA5805703,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC00010200727000004040C,expire,False
1,medical equipment,None182,USA5904658,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,DRX-ASCEND SYSTEM Accessories,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC000102007320000259259C,expire,False
2,medical equipment,13-1641VASL,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000133133C,expire,False
3,medical equipment,13-1641VASL-3,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000134134C,expire,False
4,medical equipment,13-1641,DEU5805277,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC0001020103400017001700C,expire,False
...,...,...,...,...,...,...,...,...,...,...
956,เครื่องมือแพทย์,NaN,T 6400399,Wondfo 2019-nCoV Antigen Test (Lateral Flow Me...,Wondfo 2019-nCoV Antigen Test (Lateral Flow Me...,RICHAGE GERRIK COMPANY LIMITED.,NaN,U1MRT0010264003991C,คงอยู่,None
694,เครื่องมือแพทย์,IZ-ABL-DE-001,67-1-2-1-0000023,Microalbumin rapid test cassette kit,"Microalbumin detection kit, microalbumin rapid...","K.Bio Science Co., Ltd.",ชุดตรวจหาไมโครอัลบูมินแบบตลับ microalbumin rap...,U1MN0001016713133930000011C,คงอยู่,True
695,เครื่องมือแพทย์,IZ-ABL-DE-025,67-1-2-1-0000023,Microalbumin rapid test cassette kit,"Microalbumin detection kit, microalbumin rapid...","K.Bio Science Co., Ltd.",ชุดตรวจหาไมโครอัลบูมินแบบตลับ microalbumin rap...,U1MN0001016713133930000022C,คงอยู่,True
698,เครื่องมือแพทย์,Product code : 5X4-0062,66-1-1-1-0013925,"Testing kit related to HIV infection, OraQuick...",Test kit related to HIV infection OraQuick Rap...,Pacific Biotech Company Limited,ขนาดการบรรจุ : 100 การทดสอบ/กล่อง,U1MR0001016613098150000011C,คงอยู่,True


In [45]:
to_translate = df[df.to_translate == True]
translated = df[df.to_translate == False]

In [21]:
from googletrans import Translator
translator = Translator()
from functools import lru_cache
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
@lru_cache(maxsize=None)
def translate_to_en(text):
    if pd.notna(text) and isinstance(text, str):
        translated_text = translator.translate(str(text), dest='en').text
        return translated_text
    else:
        return text
def prep_translate(df, col):
    cached_en = {}
    unique_vals = df[col].unique()
    print("API Calls: ", len(unique_vals))
    for val in tqdm(unique_vals):
        cached_en[val] = translate_to_en(val)
    return cached_en


def update_cache(val):
    translation = translate_to_en(val)
    cached_en[val] = translation
    return val, translation



In [46]:
to_translate.shape

(49, 10)

In [47]:
cached = prep_translate(to_translate, 'Product details')

API Calls:  26


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:05<00:00,  4.37it/s]


In [48]:
to_translate['Device Description'] = to_translate['Product details'].map(lambda x: translate_to_en(x))

In [49]:
th_fda = pd.concat([translated, to_translate])
th_fda

,Product_type,CAT_NO,Cert_License,Device Name,Device Description,Company,Product details,New_Codes,Status,to_translate
0,medical equipment,None50,USA5805703,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC00010200727000004040C,expire,False
1,medical equipment,None182,USA5904658,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,DRX-ASCEND SYSTEM Accessories,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC000102007320000259259C,expire,False
2,medical equipment,13-1641VASL,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000133133C,expire,False
3,medical equipment,13-1641VASL-3,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000134134C,expire,False
4,medical equipment,13-1641,DEU5805277,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC0001020103400017001700C,expire,False
...,...,...,...,...,...,...,...,...,...,...
693,เครื่องมือแพทย์,KNC-001-6,66-1-2-1-0000690,Microalbuminuria Rapid Test,"Microalbuminuria Rapid Test kit, 6 test kits/box",K.Bio Science Company Limited,ชุดทดสอบ Microalbuminuria Rapid Test จำนวน 6 ช...,U1MN0001016612903240000033C,คงอยู่,True
694,เครื่องมือแพทย์,IZ-ABL-DE-001,67-1-2-1-0000023,Microalbumin rapid test cassette kit,Microalbumin rapid test cassette detection kit...,"K.Bio Science Co., Ltd.",ชุดตรวจหาไมโครอัลบูมินแบบตลับ microalbumin rap...,U1MN0001016713133930000011C,คงอยู่,True
695,เครื่องมือแพทย์,IZ-ABL-DE-025,67-1-2-1-0000023,Microalbumin rapid test cassette kit,Microalbumin rapid test cassette detection kit...,"K.Bio Science Co., Ltd.",ชุดตรวจหาไมโครอัลบูมินแบบตลับ microalbumin rap...,U1MN0001016713133930000022C,คงอยู่,True
698,เครื่องมือแพทย์,Product code : 5X4-0062,66-1-1-1-0013925,"Testing kit related to HIV infection, OraQuick...",Packing size: 100 tests/box,Pacific Biotech Company Limited,ขนาดการบรรจุ : 100 การทดสอบ/กล่อง,U1MR0001016613098150000011C,คงอยู่,True


In [50]:
th_fda['Status'] = th_fda['Status'].map(lambda x: translate_to_en(x))

In [52]:
th_fda.to_excel("TH FDA MDR 29jan24.xlsx")

,Product_type,CAT_NO,Cert_License,Device Name,Device Description,Company,Product details,New_Codes,Status,to_translate
0,medical equipment,None50,USA5805703,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC00010200727000004040C,expire,False
1,medical equipment,None182,USA5904658,AEC ELECTRONICS FOR DR generators\nAEC ELECTRO...,DRX-ASCEND SYSTEM Accessories,"Carestream Health (Thailand) Co., Ltd.",AEC ELECTRONICS FOR DR GENERATORS,U1MC000102007320000259259C,expire,False
2,medical equipment,13-1641VASL,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000133133C,expire,False
3,medical equipment,13-1641VASL-3,DEU5601076,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC000102010340000134134C,expire,False
4,medical equipment,13-1641,DEU5805277,Bipolar cable for US-Generators\nBipolar cable...,,"Astras Medical Co., Ltd.",Bipolar cable for US-Generators,U1MC0001020103400017001700C,expire,False
...,...,...,...,...,...,...,...,...,...,...
956,medical equipment,NaN,T 6400399,Wondfo 2019-nCoV Antigen Test (Lateral Flow Me...,Wondfo 2019-nCoV Antigen Test (Lateral Flow Me...,RICHAGE GERRIK COMPANY LIMITED.,NaN,U1MRT0010264003991C,remain,True
694,medical equipment,IZ-ABL-DE-001,67-1-2-1-0000023,Microalbumin rapid test cassette kit,"Microalbumin detection kit, microalbumin rapid...","K.Bio Science Co., Ltd.",Microalbumin rapid test cassette detection kit...,U1MN0001016713133930000011C,remain,True
695,medical equipment,IZ-ABL-DE-025,67-1-2-1-0000023,Microalbumin rapid test cassette kit,"Microalbumin detection kit, microalbumin rapid...","K.Bio Science Co., Ltd.",Microalbumin rapid test cassette detection kit...,U1MN0001016713133930000022C,remain,True
698,medical equipment,Product code : 5X4-0062,66-1-1-1-0013925,"Testing kit related to HIV infection, OraQuick...",Test kit related to HIV infection OraQuick Rap...,Pacific Biotech Company Limited,Packing size: 100 tests/box,U1MR0001016613098150000011C,remain,True
